# Grokking Lisp

## Common Lisp Bootcamp

### Syntax Overview

Everything in Lisp is an object--and all objects are represented by S-Expressions. All objects in Lisp have a return value when evaluated.

An S-Expression is either an atom or a list.

In Lisp, lists of one or more elements are implemented as Cons-Cells. Atoms are any object which is not a cons.

#### Examples of atoms

In [1]:
94 ; Numbers

94

In [2]:
#||
Quoted symbols. The quote symbol ' suppresses evaluation of
the quoted symbol; It switches an expression from code to
data. The quote symbol ' is a short form for a full quote
expression.
||#

'a-quoted-symbol

A-QUOTED-SYMBOL

In [3]:
#\greek_small_letter_lamda ; Character literals

#\GREEK_SMALL_LETTER_LAMDA

In [4]:
"This is a string" ; Strings

"This is a string"

In [5]:
nil ; The empty list -- logical falsity

NIL

In [6]:
'() ; The empty list -- logical falsity

NIL

In [7]:
() ; The empty list -- logical falsity

NIL

In [8]:
t ; logical truth -- ANYTHING other than nil is truthy

T

In [11]:
'(this is a list of symbols)

(THIS IS A LIST OF SYMBOLS)

In [12]:
;; Long quote form. This could be also written as:
;; '(this is another list of symbols)

(quote (this is another list of symbols))

(THIS IS ANOTHER LIST OF SYMBOLS)

In [13]:
'another-quoted-symbol

ANOTHER-QUOTED-SYMBOL

#### Lists

Lists in lisp are enclosed in parenthesis (, ). Each nested object inside a list is a full S-Expression. Non quoted lists are treated as code by Lisp when evaluated; These are called "list forms."

Notice that:

* A form starts off with a function, macro or special operator, typically represented by a symbol.

* The rest of the form is made up of parameters passed to that function, macro or special operator.

* You can have other forms as parameters, and the innermost seem to get evaluated first; their *results* are then used as parameters to the form the next level up the hierarchy. This syntax is called "Polish prefix notation"---the operator comes first.

In [14]:
(+ 10 20 (* 30 2))

90

In [15]:
(princ "Hello, from Lisp!")

Hello, from Lisp!

"Hello, from Lisp!"

In [16]:
(loop for item in '(this list will get printed in titlecase) do (format t "~@(~A~) " item))

This List Will Get Printed In Titlecase 

NIL

In [19]:
(loop for item in '(2 12 7 37 138 5 93)
      do (format t "~a " (* item 7)))

14 84 49 259 966 35 651 

NIL

In [20]:
(+ 1 1)

2

In [21]:
(apply #'+ 1 2 3 4 '(5 6))

21

### Expressions, Parenthesis and Return Values

If the Lisp implementation can't read what you've entered as a valid S-Expression, you will normally be dropped into the Lisp debugger, the same way that it would if it tried and failed to evaluate. Destructive, in-place operations are by convention named with a prepended "N", to make it obvious that they have side-effects.

In [22]:
;; first define a couple of variables
(defvar *test-list-a* (list 1 2 3))
(defvar *test-list-b* (list 'd 'e 'f))

;; append returns a new list from its arguments
(append *test-list-a* *test-list-b*)

(1 2 3 D E F)

In [24]:
;; you can see that the original lists haven't changed
*test-list-a*

(1 2 3)

In [25]:
*test-list-b*

(D E F)

In [26]:
#||
now let's do a destructive operation, NCONC (i.e., in-place list concatenation). This will change one of the original lists passed to the NCONC function.
||#

(nconc *test-list-a* *test-list-b*)

(1 2 3 D E F)

In [27]:
;; The variable *test-list-a* in the first argument to NCONC
;; is changed by appending the value stored in the second
;; argument *test-list-b* to it. The value in *test-list-b*
;; is unchanged.

*test-list-a*

(1 2 3 D E F)

In [28]:
*test-list-b*

(D E F)

#### Return values of S-Expressions

S-Expressions in Common Lisp always return a value.

A function call returns the value of the last form in its body as the value of the entire function.

In [29]:
;; This is a typical anonymous function call; The last form
;; in its body is (+ x x), so the function call returns
;; (+ 2 2) => 4

((lambda (x) (+ x x)) 2)

4

In [30]:
;; In this function, the return result of (+ x x) is not
;; assigned, so it is essentially lost; The function body
;; moves on to the next form, (* x x), which is the last
;; form of this function body. Therefore, the function call
;; only returns (* 10 10) => 100

((lambda (x) (+ x x) (* x x)) 10)

100

In [32]:
;; In this function, we capture the return values of both
;; (+ x x) and (* x x), as the lexical variables SUM and
;; PRODUCT. Using VALUES, we can return multiple values 
;; from a form instead of just one.

((lambda (x)
         (let (
               (sum (+ x x)) 
               (product (* x x))) 
              (values sum product))) 10)

;; In the lisp REPL this returns:
;; 20
;; 100

;; But for some reason, we only see the '20' being returned
;; in this notebook REPL interface.

20

In [6]:
;; Calling VALUES without anything gives us an expression
;; with no return result

(values)

NIL

##### Self-evaluating objects

A subset of Atoms are called "self-evaluating objects". These objects return themselves as their value.

In [7]:
"a string" ; Strings are self-evaluating

"a string"

In [8]:
#\greek_small_letter_lamda ; Characters self-evaluate

#\GREEK_SMALL_LETTER_LAMDA

In [9]:
421 ; Numbers are self-evaluating

421

In [10]:
#x2A ; Even Hex-value numbers self-evaluate

42

In [11]:
#*1001 ; Bit-vectors also self-evaluate

#*1001

##### Cons cells

Non-empty lists (ordered collections of S-Expressions, surrounded by parenthesis) are read as cons-cells. Be careful to distinguish the representation from the implementation.

In [12]:
;; this:
'(a b c)

;; is the same as this:
'(a
  b
  c)

;; and this:
(list 'a 'b 'c)

(A B C)

## Lists, Cons-Cells, and Memory

In Common Lisp, S-Expressions are defined by their implementation and their representation is only treated as an interface to the underlying objects.

A list only 'conses' as long as there are values to be consed.

A cons-cell is a pair of pointers, the 'car' (Contents of Address Register) and the 'cdr' (Contents of Decrement Register).

The 'car' is usually a pointer to a value, while the 'cdr' can be a pointer to the 'car' of another cons-cell, a pointer to NIL, or (as in the case of a dotted-pair) a pointer to another value.

### Examples

In [13]:
(list)

NIL

In [14]:
(list 'a)

(A)

In [15]:
(list 'a nil)

(A NIL)

In [16]:
(cons 'a nil)

(A)

In [17]:
;; this:
(list 'a 'b 'c)

;; is the same as this:
(cons 'a (cons 'b (cons 'c nil)))

(A B C)

In [18]:
;; While this:
(list 'a 'nil)

;; is the same as this:
(cons 'a (cons nil nil))

(A NIL)

### Dotted-pairs

The end of a chain of cons-cells normally terminates in NIL, but the 'cdr' of a cons-cell can point to a value too. We can use dot notation in this case.

In [19]:
;; For example,
(cons 'a 'b)

;; can be abreviated with dot notation as
'(a . b)

(A . B)

In [20]:
#||
A list of such dotted-pairs is called an "association list" or "alist" and is useful for storing key/value pairs.
||#

'((a . b) (c . d) (e . f))

((A . B) (C . D) (E . F))

### Cons-cells and Dotted-pairs

Memory use can be optimized using alists instead of plists. A dotted-pair conses only once, while a two item list that contains the same information conses twice.

In [21]:
;; The code below creates three cons-cells, the quoted
;; symbols 'A, 'B, and 'C each in the 'car' of their own
;; cons-cell
(list 'a 'b 'c)

;; This is equivaluent to:
(cons 'a (cons 'b (cons 'c nil)))

(A B C)

In [22]:
;; Using dot notation we can get the same result as:
'(a . (b . (c . nil)))

(A B C)

In [23]:
;; Which is also equivalent to:
'(a b c . nil)

;; Or:
'(a b c)

(A B C)

## Symbols and Namespaces